# Monitoring with alibi_detect

If needed install:

In [2]:
!pip install matplotlib~=3.9.2 seaborn~=0.13.2 numpy~=1.26.4

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/charme/Documents/Gitub Repositories/MLOps_AMMI_2024/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  File "/var/folders/ks/w6y0t6b552bdg88w8kq5g_xc0000gn/T/ipykernel_59750/2789138111.py", line 1, in <module>
    get_ipython().system('pip install matplotlib~=3.9.2 seaborn~=0.13.2 numpy~=1.26.4')
  File "/Users/charme/Documents/Gitub Repositories/MLOps_AMMI_2024/.venv/lib/python3.12/site-packages/ipykernel/zmqshell.py", line 657, in system_piped
  File "/Users/charme/Documents/Gitub Repositories/MLOps_AMMI_2024/.venv/lib/python3.12/site-packages/IPython/utils/_process_posix.py", line 125, in system
ModuleNotFoundError: No module named 'pexpect'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/charme/Documents/Gitub Repositories/MLOps_AMMI_2024/.venv/lib/python3.12/site-packages/pygments/styles/__init__.py", line 45, in get_

## Introduction

Illustrating the need to monitor metrics at various window sizes to catch performance degradation as soon as possible.

Imagine we are logging/saving the F1 metrics every day fo 35 days. To mimic this, we will output random numbers in predefined intervals for every hour of the 35 days.

- For the first 20 days we will use the interval [94, 98)
- For the next 5 days we will use the interval [92, 96)
- For the next 5 days we will use the interval [88, 96)
- For the last 5 days we will use the interval [86, 92)

Please note how the interval for random generating changes, this is how we mimic a drift in the data.

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_theme()

# Generate data
hourly_f1 = list(np.random.randint(low=94, high=98, size=24*20)) + \
            list(np.random.randint(low=92, high=96, size=24*5)) + \
            list(np.random.randint(low=88, high=96, size=24*5)) + \
            list(np.random.randint(low=86, high=92, size=24*5))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/charme/Documents/Gitub Repositories/MLOps_AMMI_2024/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  File "/var/folders/ks/w6y0t6b552bdg88w8kq5g_xc0000gn/T/ipykernel_59750/3367282931.py", line 1, in <module>
    import matplotlib.pyplot as plt
ModuleNotFoundError: No module named 'matplotlib'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/charme/Documents/Gitub Repositories/MLOps_AMMI_2024/.venv/lib/python3.12/site-packages/pygments/styles/__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/charme/Documents/Gitub Repositories/MLOps_AMMI_2024/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 2168, in showtraceback
  File "/U

##### **TODO:** Compute the **cumulative mean** of the F1 values given in `hourly_f1`.

For example for an array `[3, 1, 2, 10]` a cumulative mean array would be: `[3, 2, 2, 4]`.

Explanation:
- at iteration 0: we use the element at 0th position in original array to calculate the mean --> 3/1=3
- at iteration 1: we use the elements at 0th and 1st position in original array to calculate the mean --> (3+1)/2=2
- at iteration 2: we use the elements at 0th, 1st and 2nd position in original array to calculate the mean --> (3+1+2)/3=2
- at iteration 3: we use the elements at 0th and up to 3rd position in original array to calculate the mean --> (3+1+2+10)/4=4

In [2]:
import numpy as np

def cumulative_mean(arr):
    return np.cumsum(arr) / np.arange(1, len(arr) + 1)

# Example usage:
hourly_f1 = np.array([3, 1, 2, 10])
cumulative_means = cumulative_mean(hourly_f1)

print(cumulative_means)


##### **TODO:** Print the average cumulative F1 on the last day.

##### **TODO:** Calculate the **daily sliding F1** of the values given in `hourly_f1`.

For example for an array [ 4,  2,  3,  8, -6, 10] a sliding average with a window of size 3 would be: [3, 4.33, 1.667, 4].

Explanation:

- at iteration 0: we use the first 3 elements to calculate the mean --> (4+2+3)/3=3
- at iteration 1: we use the elements at 1st,2nd,3rd position in original array to calculate the mean --> (2+3+8)/3=4.333
- at iteration 2: we use the elements at 2nd,3rd,4th position in original array to calculate the mean --> (3+8-6)/3=1.667
- at iteration 3: we use the elements at 3rd,4th,5th position in original array to calculate the mean --> (8-6+10)/3=4

##### **TODO:** Print the average sliding F1 on the last day.

##### **TODO:** Plot the cumulative and sliding values on a graph.

- Y-axes can be limited to `[80,100]`.

- Optionally, add a threshold line at `y=90`

## Data

In [ ]:
import json
import pandas as pd
from urllib.request import urlopen

##### **TODO:**
1. Read the `projects` and `tags` files and join them into a `combined` dataframe.
2. Create a new column called `text` where you will place the `title` and `description` joined with a space character -- `{title} {description}`.
3. Drop the `title` and `description` columns.
4. Initialize the `great-expectations` PandasDataset with the previously created `combined` df.

We will use this dataframe as our reference/original data, data that we used to train our ML models. Not to be mistaken with the "real-world" data received in prod

## Univariate

Drift detection on univariate data.


### Kolmogorov-Smirnov (KS) test

KS test for detecting data drift on input sequence length. We can even monitor aspects of our data that aren't necessarily inputs to the model (ex. length of input text).

In [ ]:
!pip install alibi_detect
!pip install pyOpenSSL==22.0.0

In [ ]:
from alibi_detect.cd import KSDrift

##### **TODO**:
1. Create a new column that will hold the number of words (e.g., separated by " ") in the `text` column
2. Plot a histogram for the first 200 rows from original dataframe

##### **TODO:** Initialize the KSDrift detector on the reference data.

##### **TODO:**
1. Plot a histogram for the first 200 rows from original dataframe (same as previous task).
2. On top of it, plot a histogram for the next 200 rows (i.e., `[200,400)`) from original dataframe

##### **TODO:** Apply the drift detector.

Given that both dataframes are actually part of the same data, there is no noticeble drift to be detected.

##### **TODO:**
1. Create new data (to represent production data) from a normal distribution with mean=30 and std=5; you can create 200 examples.
2. Plot a hist of this data on top of the hist of the reference data (first 200 rows).

##### **TODO:** Apply the drift detector to the reference data and the normal distributed (production) data.

### Chi-squared test

Detecting drift on categorical variables (can be used for data or target drift).


In [21]:
from alibi_detect.cd import ChiSquareDrift

##### **TODO:**
1. Add a new column to the reference data called `token_count`. This column should have the value `small` if the `num_tokens` is less or equal than 10; `medium` if in between (10,25] and `large` if larger than 25.
2. Plot the first 200 data of reference data as a histogram.

##### **TODO:** Initialize the drift detector

##### **TODO:**
1. Take the next 200 data rows from original data, and plot it over the reference data (first 200 rows).
2. Apply the drift detector.

##### **TODO:**
1. Create production data, where you have 80 times `small`, 40 times `medium` and 80 times `large`.
2. Plot the histogram of productions data on top of reference data (first 200 rows).
3. Apply drift detector.